In [5]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

### Feature Engineering for Customer Lifetime Value (CLV) Modeling

In [6]:
transactions = pd.read_csv("../data/raw/transaction_data.csv")
print(f"Transactions data shape: {transactions.shape}")
transactions.head()

Transactions data shape: (2595732, 12)


,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [7]:
transactions.dtypes

household_key          int64
BASKET_ID              int64
DAY                    int64
PRODUCT_ID             int64
QUANTITY               int64
SALES_VALUE          float64
STORE_ID               int64
RETAIL_DISC          float64
TRANS_TIME             int64
WEEK_NO                int64
COUPON_DISC          float64
COUPON_MATCH_DISC    float64
dtype: object

In [8]:
transactions.isnull().sum()

household_key        0
BASKET_ID            0
DAY                  0
PRODUCT_ID           0
QUANTITY             0
SALES_VALUE          0
STORE_ID             0
RETAIL_DISC          0
TRANS_TIME           0
WEEK_NO              0
COUPON_DISC          0
COUPON_MATCH_DISC    0
dtype: int64

In [14]:
# Get the current date (max date in the dataset plus 1 day)
current_date = transactions["DAY"].max()+1

#RFM table
rfm = transactions.groupby("household_key",as_index=False).agg({
    "DAY": lambda x : (current_date - x.max()),
    "BASKET_ID": "nunique",
    "SALES_VALUE": "sum"
}).rename(columns={
    "DAY": "RECENCY",
    "BASKET_ID": "FREQUENCY",
    "SALES_VALUE": "MONETARY"
})

# Add age of the customer in days
customers_first_purchase = transactions.groupby("household_key",as_index=False)["DAY"].min()
rfm['HOUSEHOLD_AGE'] = current_date - customers_first_purchase["DAY"]

In [15]:
print(f"RFM data shape: {rfm.shape}")
rfm.head()

RFM data shape: (2500, 5)


,household_key,RECENCY,FREQUENCY,MONETARY,HOUSEHOLD_AGE
0,1,6,86,4330.16,661
1,2,44,45,1954.34,609
2,3,9,47,2653.21,599
3,4,85,30,1200.11,608
4,5,9,40,779.06,627


In [16]:
rfm.to_csv("../data/processed/rfm.csv",index=False)